## Selecteren van data uit PostgreSQL tabel met Python script

In Python kunnen SQL queries uitgevoerd worden via de module _psycopg2_. Om te controleren of de gegevens goed ingelezen zijn, gaan we in een Python script een aantal selecties uitvoeren op de tabel _meteostation_. We gaan eerst alle rijen uit de tabel _meteostation_ ophalen en printen naar het scherm. Daarna gaan we het aantal rijen in de tabel _meteostation_ tellen. 

Het script bestaat uit de volgende stappen:

1. Inladen modules
2. Openen database connectie
3. Opstellen SQL statement om alle rijen te selecteren
4. Uitvoeren SQL statement 
5. Per rij:
   * Ophalen waardes uit rij
   * Print waardes naar scherm
6. Sluiten van database

In [ ]:
# 1. Inladen modules
import os       # Operating system interface
import psycopg2 # PostgreSQL interface

# 2. Openen database connectie
conn = psycopg2.connect("host=localhost dbname=meteodb user=postgres password=postgres port=5432")
cur = conn.cursor()

# 3. Opstellen SQL statement om alle rijen te selecteren
sql_select = 'select * from meteostation'
  
# 4. Uitvoeren SQL statement  
cur.execute(sql_select)

# 5. Per rij
for row in cur.fetchall() :

  # Ophalen waardes uit rij
  id = row[0]
  hoogte = row[1]
  naam = row[2]
  eigenaar = row[3]
  
  # Print waardes naar scherm
  print('id: ' + str(id))
  print('hoogte: ' + str(hoogte))
  print('naam: ' + str(naam))
  print('eigenaar: ' + str(eigenaar))

# 6: Sluiten van database
conn.close()

## Exporteren tabel uit PostgreSQL naar csv-bestand

Het is natuurlijk ook mogelijk om de rijen die we met Python ophalen uit de database op te slaan in een bestand. We gaan de gegevens uit de tabel _meteostation_ exporteren naar een csv-bestand waarbij de x-coördinaat en de y-coördinaat in verschillende kolommen worden opgeslagen. Om de X- en y-coördinaat uit de PostGIS geometrie te halen worden de functies [ST_X](https://postgis.net/docs/ST_X.html) en [ST_Y](https://postgis.net/docs/ST_Y.html) gebruikt. De kolomwaardes worden in het csv-bestand gescheiden door een semicolon (;). \\n wordt gebruikt om naar de volgende regel te gaan.

Het script bestaat uit de volgende stappen:

1. Inladen modules
2. Openen database connectie
3. Maken van nieuw csv-bestand en open voor schrijven ('w')
4. Ophalen van rijen uit database
5. Per rij: 
    * Kolomwaardes ophalen uit rij
    * Regel met waardes schrijven naar csv-bestand
6. Sluiten csv-bestand
7. Sluiten van database

In [ ]:
# 1. Inladen modules
import os       # Operating system interface
import psycopg2 # PostgreSQL interface

# 2. Openen database connectie
conn = psycopg2.connect("dbname=meteo user=postgres password=postgres")
cur = conn.cursor()

# 3: Maken van nieuw csv-bestand en open voor schrijven ('w')
csv_file = open('meteo_stations_from_database.csv','w')

# 4. Ophalen van rijen uit database
sql = 'select id, hoogte, naam, ST_X(geom), ST_Y(geom) from meteostation'
cur.execute(sql)
rows = cur.fetchall()

# 5. Per rij:  
for row in rows :

    # Kolomwaardes ophalen uit rij
    id = str(row[0])
    hoogte = str(row[1])
    naam = str(row[2])
    x = str(row[3])
    y = str(row[4])

    # Regel met waardes schrijven naar csv-bestand
    csv_file.write(id + ';'+ hoogte + ';' + naam + ';' + x + ';' + y + '\n')

# 6: Sluiten csv-bestand
csv_file.close()

# 7. Sluiten van database
conn.close()

## Exporteren tabel uit PostgreSQL naar csv-bestand via pandas

Het is ook mogelijk de gegevens uit de tabel _meteostation_ exporteren naar een csv-bestand met behulp van de _pandas_ module. Ook in dit geval worden de x-coördinaat en de y-coördinaat in verschillende kolommen  opgeslagen. Om de X- en y-coördinaat uit de PostGIS geometrie te halen worden de functies [ST_X](https://postgis.net/docs/ST_X.html) en [ST_Y](https://postgis.net/docs/ST_Y.html) gebruikt. De kolomwaardes worden in het csv-bestand gescheiden door een semicolon (;). 

Het script bestaat uit de volgende stappen:

1. Inladen modules
2. Openen database connectie
3. Opstellen van SQL query
4. Selecteer rijen uit database in dataframe
5. Schrijf datframe naar csv-bestand
6. Sluiten database


In [1]:
# 1. Inladen modules
import os           # Operating system interface
import psycopg2     # PostgreSQL interface
import pandas.io.sql as psql # Dataframe interface

# 2. Open database connectie
conn = psycopg2.connect("dbname=meteo user=postgres password=postgres")
cur = conn.cursor()

# 3. Opstellen van SQL query
sql = 'select id, hoogte, naam, ST_X(geom), ST_Y(geom) from meteostation'

# 4. Selecteer rijen uit database in dataframe
df = psql.read_sql(sql, conn)

# 5. Schrijf datframe naar csv-bestand
df.to_csv('meteo_stations_from_database.csv',sep=';')

# 6. Sluiten database
conn.close()

c:\Software\Python311\Lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Uitvoeren spatial join met geopandas {#odwygOmTzXx}

De Python module [_geopandas_](https://geopandas.org/) kan gebruikt worden om ruimtelijke bewerkingen uit te voeren op dataframes in Python. In het volgende script gaan we per meteostation bepalen in welke provincie het ligt. De bewerking die we gaan uitvoeren, kunnen we ook in SQL op de database doen met de volgende query:

```{sql, eval=F}
SELECT meteostation.naam, provincie.naam, meteostation.geom 
FROM meteostation,
,    province
WHERE ST_Intersects(provincie.geom, meteostation.geom)

```

De module _geopandas_ kan geïnstalleerd worden met *pip install geopandas*. Zorg er wel voor dat de afhankelijke modules _shapely_, _fiona_, _pyproj_ en _rtree_ geïnstalleerd zijn. De wheels met deze modules kunnen [hier](https://www.lfd.uci.edu/~gohlke/pythonlibs/) gedownload worden. Uitgebreide toelichting op installatie proces is [hier](https://geopandas.org/getting_started/install.html) te vinden. Een overzicht van de mogelijkheden van _geopandas_ is [hier](https://geopandas.org/gallery/index.html) te vinden.  

Het script bestaat uit de volgende stappen:

1. Inladen modules
2. Importeren csv-bestand meteostations in dataframe
3. Maken een geodataframe met meteostations
4. Importeren shp-bestand provincies in geodataframe
5. Coordinaattransformatie naar EPSG:4326
6. Koppelen provincie aan meteostation met spatial join
7. Selecteren de kolommen die je wilt behouden
8. Opslaan resultaat als geopackage-bestand

In [ ]:
# 1. Import modules
import pandas as pd
import geopandas as gpd
import numpy as np

# 2. Importeren csv-bestand meteostations in dataframe
df_meteostations = pd.read_csv('data/meteo_stations.csv',sep=';')

# 3. Maken een geodataframe met meteostations
gdf_meteostations = gpd.GeoDataFrame(df_meteostations, geometry=gpd.points_from_xy(df_meteostations['lon'],df_meteostations['lat']),crs='EPSG:4326')

# 4. Importeren shp-bestand provincies in geodataframe
gdf_provincies = gpd.read_file("data/2018_provinciegrenzen_met_water.shp")

# 5. Coordinaattransformatie naar EPSG:4326
gdf_provincies = gdf_provincies.to_crs("EPSG:4326")

# 6. Koppelen provincie aan meteostation met spatial join
gdf_spatial_join = gpd.sjoin(gdf_meteostations, gdf_provincies, how='inner')

# 7. Selecteren de kolommen die je wilt behouden
gdf_resultaat = gdf_spatial_join[['meteostationid', 'name', 'geometry', 'Provincien']]
print(gdf_resultaat)

# 8. Opslaan resultaat als geopackage-bestand
gdf_resultaat.to_file("deleteme.gpkg", driver="GPKG")

## fiona, shapely, rasterio {#kdwygOmTzXx}

De module [_fiona_](https://pypi.org/project/Fiona/) is een vector API gebouwd op _GDAL_ en kan gebruikt worden voor het lezen en schrijven van GIS vectordata en maakt in tegenstelling tot het object geörienteerde _GDAL_ gebruik van files, directories, mappings en iteratoren.   

De module [_rasterio_](https://rasterio.readthedocs.io/en/latest/index.html) is een raster API gebouwd op _GDAL_ en maakt het werken met raster datasets eenvoudiger en meer in de lijn met andere Python modules.

De module [_shapely_](https://pypi.org/project/Shapely/) kan gebruikt worden voor het manipuleren en analyseren van geometrische objecten in het Cartesisch vlak. _shapely_ houdt geen rekening met coördinaatsystemen en dataformaten. Hiervoor moeten andere modules gebruikt worden.